In [4]:
from dotenv import load_dotenv
import dspy
import pandas as pd

from agentic_system.agents import CFEfficacyAgent
from agentic_system.litl_data.litl_utils import load_efficacy_devset

load_dotenv("../.env")
dspy.configure(lm=dspy.LM("gemini/gemini-2.5-flash-lite", temperature=0.5, cache=True))

In [5]:
efficacy_devset = load_efficacy_devset()
efficacy_devset = efficacy_devset * 5
len(efficacy_devset)

210

In [6]:
from dspy.evaluate import Evaluate


def efficacy_accuracy(example, pred, trace=None):
    "Calculate error from prediction"
    abs_error = abs(example.cf_efficacy - float(pred.predicted_efficacy))
    accuracy = 1 - abs_error
    return accuracy


# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(
    devset=efficacy_devset[:2],
    num_threads=10,
    display_progress=True,
    provide_traceback=True,
)

efficacy_agent = CFEfficacyAgent(max_iters=5)
evaluation = evaluator(efficacy_agent, metric=efficacy_accuracy)
evaluation

Average Metric: 0.13 / 2 (6.6%): 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]

2025/09/15 15:27:09 INFO dspy.evaluate.evaluate: Average Metric: 0.13114542588046052 / 2 (6.6%)


EvaluationResult(score=6.56, results=<list of 2 results>)

In [7]:
evaluation.results[0]

(Example({'compound_name': 'Luminespib', 'cf_efficacy': 0.9384660045676458}) (input_keys={'compound_name'}),
 Prediction(
     trajectory={'thought_0': "The user wants to estimate the efficacy of Luminespib in reversing the activated cardiac fibroblast phenotype. The efficacy is defined as a metric between 0 and 1 based on a custom in vitro assay. To determine this, I need to find information about Luminespib's bioactivities, specifically any data related to its effect on fibroblasts or related cellular processes. I will start by searching for Luminespib in ChEMBL to get its ChEMBL ID, and then use that ID to retrieve its bioactivity summary.", 'tool_name_0': 'CHEMBL__search_chembl_id', 'tool_args_0': {'query': 'Luminespib'}, 'observation_0': "Found 2 compound(s) matching 'Luminespib': \n - CHEMBL3137334 (LUMINESPIB MESYLATE)\n - CHEMBL252164 (LUMINESPIB)", 'thought_1': 'I have found two ChEMBL IDs for Luminespib: CHEMBL3137334 and CHEMBL252164. To get the bioactivity summary, I should